# Modélisation et expression des forces

In [1]:
from abc import ABC, abstractmethod
import numpy as np

class Force(ABC):
    """Classe abstraite de modélisation d'une force."""

    @property
    def G(self):
        """Constante universelle de gravitation."""
        return 6.67430e-20  # km^3 kg^−1 s^−2

    @property
    def M(self):
        """Masse de la Terre."""
        return 5.9736e24  # kg

    @property
    def R(self):
        """Rayon moyen volumétrique de la Terre."""
        return 6371  # km

    @property
    @abstractmethod
    def name(self):
        """Nom de la force sous forme de string."""
        raise NotImplementedError(self.name)
    
    @property
    @abstractmethod
    def formula(self):
        """Expression de la force au format LaTeX."""
        raise NotImplementedError(self.formula)
    
    @abstractmethod
    def acceleration(self, u, t):
        """Accélération de la force appliquée au vecteur spécifié."""
        raise NotImplementedError(self.acceleration)
        
    def acc_norm(self, u, t=0):
        """Norme de l'accélération appliquée au vecteur spécifié."""
        a = self.acceleration(u, t)
        return np.linalg.norm(a)
    
    def __repr__(self):
        """Représentation de la force sous forme de string."""
        return self.name
    
    def _repr_latex_(self):
        """[Jupyter] Représentation de la force au format LaTeX."""
        return fr"{self.name} : $\displaystyle {self.formula}$"


class ForceSet():
    """Classe de jeu de forces à appliquer à une particule."""
    def __init__(self, forces):
        """Instancie la classe à partir d'une liste de forces."""
        self.forces = forces

    def derivee(self, u, t):
        """Calcule la dérivée à partir d'un vecteur de coordonnées.
        
        Dans un repère cartésien tridimensionnel, le vecteur de
        coordonnées est `[x, y, z, dx, dy, dz]`.
        """
        # vecteur des coordonnées sources
        x, y, z, vx, vy, vz = u
        # les vitesses sont les dérivées des positions
        dx, dy, dz = vx, vy, vz
        # les acclélérations sont définies par les EDO de chaque force
        dvx, dvy, dvz = np.sum([force.acceleration(u, t)
                                for force in self.forces], axis=0)
        # renvoi des dérivées
        return np.array([dx, dy, dz, dvx, dvy, dvz])

    def magnitude(self, u):
        """Calcul des ordres de grandeur des forces à des coordonnées."""
        return {str(force): np.log10(force.acc_norm(u)) for force in forces}

    def solve(self, t_start, t_stop, t_step, coords):
        """Résolution d'équation différentielle par la méthode de Runge-Kutta.

        Attributs:
            t_start: valeur de départ de la variable temporelle
            t_stop: valeur d'arrivée de la variable temporelle
            t_step: pas temporel
            coords: coordonnées initiales, valeur initiale de la variable libre
         """
        # Création du tableau temps
        num_points = int((t_stop - t_start) / t_step) + 1
        t = np.linspace(t_start, t_stop, num_points)
        # Initialisation du tableau solution
        v = np.empty((len(coords), num_points))
        # Condition initiale
        v[:, 0] = coords
        # Boucle for des variables de la méthode
        for i in range(num_points - 1):
            d1 = self.derivee(v[:, i], t[i])
            d2 = self.derivee(v[:, i] + t_step / 2 * d1, t[i] + t_step / 2)
            d3 = self.derivee(v[:, i] + t_step / 2 * d2, t[i] + t_step / 2)
            d4 = self.derivee(v[:, i] + t_step * d3,  t[i] + t_step)
            v[:, i + 1] = v[:, i] + t_step / 6 * (d1 + 2*d2 + 2*d3 + d4)
        # Renvoi des tableaux des temps et des solutions
        return t, v

    def __repr__(self):
        """Représentation du jeu de forces sous forme de string."""
        return fr"ForceSet[{', '.join(str(force) for force in self.forces)}]"    

Force de gravitation (Terre) :
$$\ddot r = -\frac{GM}{\mathbf{r}^3} \mathbf{r}$$

In [2]:
class Earth_Gravity(Force):
    """Force de gravitation terrestre."""

    @property
    def name(self):
        """Nom de la force sous forme de string."""
        return "Force de gravitation terrestre"
    
    @property
    def formula(self):
        """Expression de la force au format LaTeX."""
        return r"-\frac{GMm}{r^3}\mathbf r"

    def acceleration(self, u, t):
        """EDO gravitation terrestre."""
        # vecteur position        
        r = np.array(u[:3])
        # Expression de l'acceleration résultante de la force
        return - self.G * self.M / np.linalg.norm(r)**3 * r

Influence du soleil :
$$\ddot r = \frac{GM\mathbf r}{s^3}\big(-\mathbf{e}_r+3\mathbf{e}_s(\mathbf{e}_s\mathbf{e}_r)\big)$$

Influence de la lune :
$$\ddot r = \frac{GM\mathbf r}{l^3}\big(-\mathbf{e}_r+3\mathbf{e}_l(\mathbf{e}_l\mathbf{e}_r)\big)$$

In [3]:
from astropy.time import Time
import astropy.units as units


class Body_Influence(Force):
    """Influence gravitationnelle relative à un autre corps."""
    def __init__(self, body_name, astropy_position_callback, t0):
        """Instanciation avec les paramètres du corps tiers concerné.
        
        Requiert le nom du corps, sa fonction position (issue du module
        `astropy`) et un temps initial.
        """
        # Le nom du corps concerné
        self.body_name = body_name
        # La fonction issue d'`astropy.coordinates` des coordonnées du corps
        self.pos = astropy_position_callback
        # Enregistre le temps initial au format `astropy.time.Time`
        self.t0 = Time(iss_data.iloc[0].datetime)

    @property
    def name(self):
        """Nom de la force sous forme de string."""
        return f"Influence gravitationnelle relative : {self.body_name}"
    
    @property
    def formula(self):
        """Expression de la force au format LaTeX."""
        # Initale du corps
        b = self.body_name[0].lower()
        # Formule avec l'initale du corps pour le vecteur unitaire directeur
        return (fr"-\frac{{GMm}}{{{b}^3}}\big(-\mathbf e_r + "
                fr"3\mathbf e_{b}(\mathbf e_{b}\mathbf e_r)\big)")

    def position_at(self, t):
        """Coordonnées cartésiennes kilométriques du corps au temps `t`."""
        # Calcul du temps absolu à partir du temps initial
        time = self.t0 + t * units.s
        # Renvoi des coordonnées (`x`, `y`, `z`)
        return self.pos(time).represent_as('cartesian').xyz.to(units.km).value

    def acceleration(self, u, t):
        """EDO Influence relative d'un corps."""
        # vecteur position        
        r = np.array(u[:3])
        b = self.position_at(t)
        er = r / np.linalg.norm(r)
        eb = b / np.linalg.norm(b)
        # Expression de l'acceleration résultante de la force
        return (self.G * self.M * r / np.linalg.norm(b)**3
                * (- er + 3 * eb * np.dot(eb, er)))

Trainée atmosphérique :
$$ \ddot r = -\frac12 C_D \frac{A}m \rho_0 e^{-h/H_0} \left(\dot r - \mathbf\omega \wedge r\right)^2 \textbf e_v $$

Le terme $(\dot r - \omega r)^2$ doit être remplacé par $(|\dot{\textbf{r}} - \boldsymbol{\omega} \times \textbf{r}|)^2$, où $\boldsymbol{\omega}$ est la vitesse angulaire de la Terre et $\textbf{r}$ est le vecteur position de l'ISS. Ce terme représente la *norme* de la vitesse relative de l'ISS par rapport à l'atmosphère. $\times$ Représente ici le produit vectoriel.

On a alors 
$$\left\lvert \dot{\mathbf{r}} - \boldsymbol{\omega} \times \mathbf{r} \right\rvert = \sqrt{(\dot{x} - \omega_y z + \omega_z y)^2 + (\dot{y} - \omega_z x + \omega_x z)^2 + (\dot{z} - \omega_x y + \omega_y x)^2}$$



In [4]:
class Atmospheric_Drag(Force):
    """Modélisation de la traînée atmosphérique."""
    def __init__(self, mass, drag_coeff, drag_area):
        self.mass = mass
        self.drag_coeff = drag_coeff
        self.drag_area = drag_area

    @property
    def name(self):
        """Nom de la force sous forme de string."""
        return "Traînée atmosphérique"
    
    @property
    def formula(self):
        """Expression de la force au format LaTeX."""
        return (r"-\frac12 C_D A \rho_0 e^{-h/H_0}"
                r"\left(\dot r - \omega r\right)^2 \textbf e_v")

    @property
    def rho_0(self):
        """Densité atmosphérique du point de référence, ici le niveau de la mer."""
        return 1.225  # kg.m^-3
    
    @property
    def H_0(self):
        """Hauteur de l'échelle de densité, ici la hauteur moyenne de la mer."""
        return 7.9  # km

    @property
    def omega(self):
        """Vecteur vitesse angulaire moyenne de la Terre."""
        return [0, 0, 7.292e-5] # rad.s^-1

    def acceleration(self, u, t):
        """EDO traînée atmosphérique."""
        # vecteurs position et vitesse        
        r, dotr = np.split(np.array(u), 2)
        # Hauteur de la station
        h = np.linalg.norm(r) - self.R
        # Vitesse relative à celle de l'atmosphère
        v_r = dotr - np.cross(self.omega, r)
        # Vecteur unitaire de v_r
        e_v = v_r / np.linalg.norm(v_r)
        # Expression de l'acceleration résultante de la force
        return (-1/2 * self.drag_coeff * self.drag_area/self.mass
                * self.rho_0 * np.exp(-h/self.H_0) * v_r.dot(v_r) * e_v)

Géopotentiel :
$$
\ddot{r} = \nabla\frac{GM}{r} + \nabla\frac{GM}{r} \frac{R^2}{r^2} \sqrt{5} \frac{1}{2}\left(3 \sin^2\varphi - 1\right) \bar{C}_{2 ,0} + \nabla\frac{GM}{r}\frac{R^2}{r^2} 3\cos^2\varphi \sqrt \frac{5}{12} \left(\bar{C}_{2,2} \cos(2\lambda) + \bar{S}_{2,2} \sin(2\lambda)\right)
$$

Simplification de la formule :
$$
\ddot r = \nabla\frac{GM}r\left[1+\frac{\sqrt5}2\frac{R^2}{r^2}\Bigg(\bar{C}_{2,0}\left(3\sin^2(\varphi)-1\right)+\sqrt3\cos^2(\varphi)\left(\bar{C}_{2,2}\cos(2\lambda)+\bar{S}_{2,2}\sin(2\lambda)\right)\Bigg)\right]
$$

Sachant :
$$
\begin{align*}
\nabla\frac{GM}r
&= GM\left[
       \frac{\mathrm d}{\mathrm dx}\left(\frac1{\sqrt{x^2+y^2+z^2}}\right)
       +\frac{\mathrm d}{\mathrm dy}\left(\frac1{\sqrt{x^2+y^2+z^2}}\right)
       +\frac{\mathrm d}{\mathrm dz}\left(\frac1{\sqrt{x^2+y^2+z^2}}\right)
     \right]\\
&= GM\left[
       -\frac{\mathbf x}{\left(x^2+y^2+z^2\right)^{\frac32}}
       -\frac{\mathbf y}{\left(x^2+y^2+z^2\right)^{\frac32}}
       -\frac{\mathbf z}{\left(x^2+y^2+z^2\right)^{\frac32}}
     \right]\\
&= -GM\left(
       \frac{\mathbf x+\mathbf y+\mathbf z}{r^3}
     \right)\\
&= - \frac{GM}{r^3}\mathbf r
\end{align*}
$$

In [5]:
from isslib.coordinates import cartesian2spherical

class Geopotential(Force):

    @property
    def C20(self):
        """Valeur du coefficient de Legendre cosinus de degré 2 d'ordre 0."""
        return -484.2e-6  # coefficient sans unité

    @property
    def C22(self):
        """Valeur du coefficient de Legendre cosinus de degré 2 d'ordre 2."""
        return 2.439261e-6  # coefficient sans unité

    @property
    def S22(self):
        """Valeur du coefficient de Legendre sinus de degré 2 d'ordre 2."""
        return -1.400266e-6  # coefficient sans unité

    @property
    def name(self):
        """Nom de la force sous forme de string."""
        return "Force géopotentielle"
    
    @property
    def formula(self):
        """Expression de la force au format LaTeX."""
        return (r"\nabla\frac{GMm}r\left[1+\frac{\sqrt5}2\frac{R^2}{r^2}"
                r"\Bigg(\bar{C}_{2,0}\left(3\sin^2(\varphi)-1\right)+"
                r"\sqrt3\cos^2(\varphi)\left(\bar{C}_{2,2}\cos(2\lambda)+"
                r"\bar{S}_{2,2}\sin(2\lambda)\right)\Bigg)\right]")

    def acceleration(self, u, t):
        """EDO géopotentiel."""
        # vecteur position        
        r = np.array(u[:3])
        # longitude et latitude stellaire
        longitude, latitude = cartesian2spherical(*r)
        # Norme de R
        n_r = np.linalg.norm(r)
        # nabla(GM/r)
        nablaGMr = - self.G *self.M / n_r**3 * r
        # Expression de l'acceleration résultante de la force
        return nablaGMr * (1 + np.sqrt(5)*self.R**2/(2*n_r**2)
                           * (self.C20 * (3 * np.sin(latitude)**2 - 1)
                              + np.sqrt(3) * np.cos(latitude)**2
                              * (self.C22 * np.cos(2*longitude)
                                 + self.S22 * np.sin(2*longitude))))

In [6]:
import isslib

# Coordonnées de l'ISS
iss = isslib.ISS_Position()
iss_data = iss.get_data().query('thrust_episode==0')
# Métadonnées utiles
iss_mass = iss.get_metadata("MASS")
iss_drag_coeff = iss.get_metadata("DRAG_COEFF")
iss_drag_area = iss.get_metadata("DRAG_AREA")
# Colonnes d'interêt
columns = ['x', 'y', 'z', 'vx', 'vy', 'vz']
# Variables initiales
ini = np.array(iss_data.iloc[0][columns], dtype=np.float64)
t0 = iss_data.iloc[0]['datetime']
# Durée de la plage temporelle
t_final = iss_data.iloc[-1]['datetime']
duration = (t_final - t0).total_seconds()
# Etapes de 4 minutes (identique aux données sources)
t_step = 240

In [7]:
import astropy.coordinates as co

# Liste des forces modélisées
forces = [Earth_Gravity(),
          Geopotential(),
          Body_Influence('Lune',co.get_moon, t0),
          Body_Influence('Soleil',co.get_sun, t0),
          Atmospheric_Drag(iss_mass, iss_drag_coeff, iss_drag_area),
         ]
# Création du jeu de forces
forceset = ForceSet(forces)

In [8]:
[6795, 0, 0]
[0, 7.65, 0]


[0, 7.65, 0]

In [9]:
np.sqrt(ini[3]**2+ini[4]**2+ini[5]**2)


7.656223304586446

In [10]:
import pandas as pd

# Calcul et affichage des ordres de grandeur
magn = forceset.magnitude(ini)
pd.DataFrame(magn.values(), index=magn.keys(), columns=["ordre de grandeur"])

,ordre de grandeur
Force de gravitation terrestre,-2.063849
Force géopotentielle,-2.063947
Influence gravitationnelle relative : Lune,-7.460468
Influence gravitationnelle relative : Soleil,-15.285848
Traînée atmosphérique,-23.935329


[TODO] Y'a un souci

In [11]:
# Calcul de l'ordre de grandeur différentiel du géopotentiel si on théorise qu'il inclut la gravitation
np.log10(np.abs(magn['Force géopotentielle'] - magn['Force de gravitation terrestre']))

-4.009732338756875

In [12]:
# Application du modèle
duration = min(duration, 24000)  # Cap for debug
t, coordinates = forceset.solve(0, duration, t_step, ini)

In [13]:
# Affichage des résultats
df = pd.DataFrame(coordinates.T, index=t, columns=ini.index if isinstance(ini, pd.Series) else None)
df # WTF ?

,0,1,2,3,4,5
0.0,-3.149866e+03,3.681707e+03,-4.765218e+03,-3.379971e+00,-6.332376e+00,-2.663563e+00
240.0,-3.708785e+03,1.929022e+03,-5.037587e+03,-1.178313e+00,-8.120260e+00,4.800275e-01
480.0,2.117639e+18,-1.328084e+20,3.394797e+19,1.764699e+16,-1.106737e+18,2.828997e+17
720.0,6.352917e+18,-3.984253e+20,1.018439e+20,1.764699e+16,-1.106737e+18,2.828997e+17
960.0,1.058819e+19,-6.640423e+20,1.697398e+20,1.764697e+16,-1.106738e+18,2.828996e+17
...,...,...,...,...,...,...
23040.0,4.000163e+20,-2.511160e+22,6.414639e+21,1.761769e+16,-1.108131e+18,2.826940e+17
23280.0,4.042445e+20,-2.537755e+22,6.482485e+21,1.761704e+16,-1.108160e+18,2.826895e+17
23520.0,4.084725e+20,-2.564351e+22,6.550330e+21,1.761639e+16,-1.108188e+18,2.826849e+17
23760.0,4.127003e+20,-2.590948e+22,6.618173e+21,1.761573e+16,-1.108217e+18,2.826802e+17


In [14]:
df2 = df[np.abs(df.x) <= 10e4]
with isslib.visualization.orbital() as fig:
    fig.add_scatter3d(x=df2.x, y=df2.y, z=df2.z, line_color='red', name="Episode 0")
    fig.update_layout(title="Modélisation de gravitation terrestre à partir de coordonnées initiales")

AttributeError: 'DataFrame' object has no attribute 'x'